In [105]:
import requests
import pandas as pd
import uuid
import time
import tqdm
import numpy as np
from sklearn.metrics import classification_report, accuracy_score, f1_score
from converters import parse_data

base_url = 'https://api-trial.cognigy.ai/new/'
api_key = 'xxx'

1. Go to https://trial.cognigy.ai/login
2. Create Flow with Intents
3. Import Intents using CSV import in Web interface
4. Build model in Web Interface
5. Add Code node to Flow Chart with following code (actions.output("intentScore:" + input.intentScore +";intent:"+input.intent);)
5. Query from here

In [88]:
def convert_data_to_cognigy_format(path, savepath):
    data, _, _ = parse_data(path)
    csv_col0 = []
    csv_col1 = []
    csv_col2 = []
    for intent_name, intent_data in data.items():    
        csv_col0.append(intent_name)
        csv_col1.append('exampleSentence')
        csv_col2.append(intent_data['question'])
        for question in intent_data['paraphrased_questions']:
            csv_col0.append(intent_name)
            csv_col1.append('exampleSentence')
            csv_col2.append(question)
    csv_col0[:2], csv_col1[:2], csv_col2[:2]
    df = pd.DataFrame({'c0': csv_col0, 'c1': csv_col1, 'c2': csv_col2})
    df.to_csv(savepath, header=None, index=None)

In [101]:
def query_api(text):
  rest_endpoint = 'https://endpoint-trial.cognigy.ai/xxx'
  rest_params = {
    "userId":"312312",
    "sessionId": uuid.uuid4().hex,
    "text": text,
    "api_key": api_key,
  }
  response = requests.post(rest_endpoint, json=rest_params)
  return response

In [102]:
def get_intent_name(response):
    splitted = response.json()['text'].split(';')
    intent_score = splitted[0]
    intent_name_raw = splitted[1]
    intent_name = intent_name_raw.split("intent:")[1].replace(" __GARBAGE__", "")
    return intent_name

In [103]:
times = []
_, X, y = parse_data('data/hwu_small_test.csv')
y_pred = []
for sent in tqdm.tqdm(X):
    start_time = time.time()
    response = query_api(sent)
    times.append(time.time() - start_time)
    y_pred.append(get_intent_name(response))

100%|██████████| 1076/1076 [10:39<00:00,  1.68it/s]


In [106]:
print(classification_report(y, y_pred))
print("Accuracy: ", accuracy_score(y, y_pred)) 
print("F1-Score: ", f1_score(y, y_pred, average='macro')) 
print(f"Mean response time: {np.mean(times)} +- {np.std(times)} sec.")

                          precision    recall  f1-score   support

             alarm_query       1.00      0.89      0.94        19
            alarm_remove       0.69      0.82      0.75        11
               alarm_set       0.80      0.63      0.71        19
       audio_volume_down       0.54      0.88      0.67         8
       audio_volume_mute       1.00      0.73      0.85        15
         audio_volume_up       0.88      0.54      0.67        13
          calendar_query       0.57      0.42      0.48        19
         calendar_remove       0.94      0.89      0.92        19
            calendar_set       0.67      0.74      0.70        19
          cooking_recipe       0.94      0.84      0.89        19
        datetime_convert       0.58      0.88      0.70         8
          datetime_query       0.71      0.63      0.67        19
        email_addcontact       0.80      1.00      0.89         8
             email_query       0.89      0.89      0.89        19
      ema

/usr/local/var/pyenv/versions/3.8.1/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/var/pyenv/versions/3.8.1/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/var/pyenv/versions/3.8.1/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [107]:
convert_data_to_cognigy_format('data/hwu_large_train.csv', 'data/hwu_large_train_cognigy.csv')

In [108]:
times = []
_, X, y = parse_data('data/hwu_large_test.csv')
y_pred = []
for sent in tqdm.tqdm(X):
    start_time = time.time()
    response = query_api(sent)
    times.append(time.time() - start_time)
    y_pred.append(get_intent_name(response))

100%|██████████| 5518/5518 [54:24<00:00,  1.69it/s]


In [109]:
print(classification_report(y, y_pred))
print("Accuracy: ", accuracy_score(y, y_pred)) 
print("F1-Score: ", f1_score(y, y_pred, average='macro')) 
print(f"Mean response time: {np.mean(times)} +- {np.std(times)} sec.")

                          precision    recall  f1-score   support

             alarm_query       0.95      0.86      0.91        94
            alarm_remove       0.90      0.87      0.89        54
               alarm_set       0.81      0.86      0.84        96
       audio_volume_down       0.73      0.82      0.78        40
       audio_volume_mute       0.84      0.74      0.78        76
         audio_volume_up       0.89      0.77      0.82        73
          calendar_query       0.64      0.41      0.50        95
         calendar_remove       0.85      0.89      0.87       102
            calendar_set       0.63      0.78      0.70        91
          cooking_recipe       0.90      0.91      0.90       107
        datetime_convert       0.72      1.00      0.84        47
          datetime_query       0.73      0.72      0.72        92
        email_addcontact       0.90      0.97      0.94        38
             email_query       0.91      0.88      0.90        93
      ema

/usr/local/var/pyenv/versions/3.8.1/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/var/pyenv/versions/3.8.1/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/var/pyenv/versions/3.8.1/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
